In [1]:
!pip install faker

  Using cached Faker-28.4.1-py3-none-any.whl.metadata (15 kB)
Using cached Faker-28.4.1-py3-none-any.whl (1.8 MB)
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/home/research/Kilsar_Sentinal/.venv/lib/python3.11/site-packages/faker'
Check the permissions.


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
import multiprocessing

# Initialize Faker
faker = Faker()
Faker.seed(0)

# Define time range (1 year)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date)

# Define lat/long range with 1 degree steps
lat_range = np.arange(-90, 91, 1)
long_range = np.arange(-180, 181, 1)

# Function to generate data for a given range of latitudes
def generate_data_for_lat_range(lat_start, lat_end):
    data = []
    for lat in np.arange(lat_start, lat_end, 1):
        print(f"Processing lat: {lat}")
        for long in long_range:
            for date in date_range:
                # Generate a fake density value
                density_N0X = faker.random_number(digits=5)
                data.append([lat, long, date, density_N0X])
    return data

# Function to divide latitude range into chunks for each core
def parallel_data_generation(lat_range, num_cores):
    lat_chunks = np.array_split(lat_range, num_cores)
    with multiprocessing.Pool(num_cores) as pool:
        results = pool.starmap(generate_data_for_lat_range, [(chunk[0], chunk[-1]+1) for chunk in lat_chunks])
    return results

if __name__ == "__main__":
    # Get the number of CPU cores
    num_cores = multiprocessing.cpu_count()
    print(f"Using {num_cores} CPU cores")

    # Generate data in parallel
    results = parallel_data_generation(lat_range, num_cores)

    # Flatten the list of results
    data = [item for sublist in results for item in sublist]

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['latitude', 'longitude', 'date', 'density_N0X'])

    # Save the dataframe to a CSV file
    df.to_csv('fake_density_data_parallel.csv', index=False)

    # Show the first few rows of the dataset
    print(df.head())


Using 96 CPU cores
Processing lat: -86Processing lat: -90Processing lat: -88Processing lat: -84Processing lat: -82Processing lat: -80Processing lat: -78Processing lat: -76Processing lat: -74Processing lat: -72Processing lat: -70Processing lat: -68

Processing lat: -66Processing lat: -64


Processing lat: -62





Processing lat: -60Processing lat: -56
Processing lat: -58Processing lat: -54Processing lat: -52
Processing lat: -50
Processing lat: -44Processing lat: -42
Processing lat: -38Processing lat: -36Processing lat: -40Processing lat: -28Processing lat: -32Processing lat: -34Processing lat: -26Processing lat: -30
Processing lat: -22
Processing lat: -24

Processing lat: -16

Processing lat: -20Processing lat: -18Processing lat: -14Processing lat: -10Processing lat: -8Processing lat: -12




Processing lat: -6Processing lat: -2Processing lat: -4


Processing lat: 4
Processing lat: 2Processing lat: 0
Processing lat: 6Processing lat: 8
Processing lat: 10Processing lat: 12
Processing lat

In [3]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
import multiprocessing

# Initialize Faker
faker = Faker()
Faker.seed(0)

# Define time range (1 year)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date)

# Define lat/long range with 0.5 degree steps
lat_range = np.arange(-89.75, 91, 0.5)
long_range = np.arange(-179.75, 181, 0.5)

# Function to generate data for a given range of latitudes
def generate_data_for_lat_range(lat_start, lat_end):
    data = []
    for lat in np.arange(lat_start, lat_end, 1):
        print(f"Processing lat: {lat}")
        for long in long_range:
            for date in date_range:
                # Generate a fake density value
                scalar_N0X = faker.random_number(digits=5)
                data.append([lat, long, date, scalar_N0X])
    return data

# Function to divide latitude range into chunks for each core
def parallel_data_generation(lat_range, num_cores):
    lat_chunks = np.array_split(lat_range, num_cores)
    with multiprocessing.Pool(num_cores) as pool:
        results = pool.starmap(generate_data_for_lat_range, [(chunk[0], chunk[-1]+1) for chunk in lat_chunks])
    return results

if __name__ == "__main__":
    # Get the number of CPU cores
    num_cores = multiprocessing.cpu_count()
    print(f"Using {num_cores} CPU cores")

    # Generate data in parallel
    results = parallel_data_generation(lat_range, num_cores)

    # Flatten the list of results
    data = [item for sublist in results for item in sublist]

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=['latitude', 'longitude', 'date', 'scalar_N0X'])

    # Save the dataframe to a CSV file
    df.to_csv('fake_scalar_data_parallel_0.5.csv', index=False)

    # Show the first few rows of the dataset
    print(df.head())


Using 96 CPU cores
Processing lat: -89.75Processing lat: -83.75Processing lat: -87.75Processing lat: -81.75Processing lat: -85.75Processing lat: -79.75Processing lat: -75.75Processing lat: -73.75Processing lat: -77.75
Processing lat: -71.75Processing lat: -67.75Processing lat: -65.75Processing lat: -69.75Processing lat: -63.75






Processing lat: -57.75
Processing lat: -55.75Processing lat: -53.75


Processing lat: -49.75Processing lat: -51.75

Processing lat: -45.75Processing lat: -47.75Processing lat: -43.75Processing lat: -41.75Processing lat: -39.75
Processing lat: -37.75Processing lat: -35.75
Processing lat: -31.75Processing lat: -33.75
Processing lat: -27.75Processing lat: -25.75Processing lat: -29.75Processing lat: -21.75

Processing lat: -23.75
Processing lat: -19.75Processing lat: -15.75
Processing lat: -17.75Processing lat: -11.75Processing lat: -13.75
Processing lat: -9.75


Processing lat: -5.75Processing lat: -7.75Processing lat: -3.75
Processing lat: -1.75Processing lat

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from multiprocessing import Pool

# Function to load and convert to GeoDataFrame
def load_data_to_gdf(file_path, lat_col='latitude', long_col='longitude', crs='EPSG:4326'):
    df = pd.read_csv(file_path)
    # Create geometry column based on lat/long
    geometry = [Point(xy) for xy in zip(df[long_col], df[lat_col])]
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    print("loaded the data", file_path)
    return gdf

# Function to merge two GeoDataFrames by lat/long and date
def merge_gdfs(gdf1, gdf2):
    # Merge on geometry (lat/long) and date
    merged_gdf = gdf1.merge(gdf2, on=['geometry', 'date'], how='inner')
    return merged_gdf

# Function to merge two GeoDataFrames using nearest join and left join on geometry
def merge_gdfs_nearest(gdf_left, gdf_right):
    # Perform a spatial join where we merge the nearest points (left join)
    merged_gdf = gpd.sjoin_nearest(gdf_left, gdf_right, how='left', distance_col='distance', max_distance=100000)
    return merged_gdf


# Parallel processing function
def parallel_merge(args):
    gdf1_chunk, gdf2 = args
    return merge_gdfs(gdf1_chunk, gdf2)

# Main process
if __name__ == "__main__":
    # Load datasets into GeoDataFrames
    gdf_density = load_data_to_gdf('fake_density_data_parallel.csv')
    gdf_scalar = load_data_to_gdf('fake_scalar_data_parallel_0.5.csv')

    # Split the larger dataframe into chunks for parallel processing
    num_cores = 4
    gdf_density_chunks = np.array_split(gdf_density, num_cores)

    # Use multiprocessing to merge the chunks
    with Pool(num_cores) as pool:
        results = pool.map(parallel_merge, [(chunk, gdf_scalar) for chunk in gdf_density_chunks])

    # Combine results into one GeoDataFrame
    merged_gdf = pd.concat(results)

    # Save the merged GeoDataFrame
    merged_gdf.to_csv('merged_density_scalar_data.csv', index=False)

    # Example of multiplying the scalar and density data
    merged_gdf['result'] = merged_gdf['density_N0X'] * merged_gdf['scalar_value']

    print(merged_gdf.head())


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from multiprocessing import Pool
import os

# Function to load a CSV file as a regular DataFrame
def load_data(file_path):
    print("Trying to read the file ", file_path)
    df = pd.read_csv(file_path)
    print(f"We have read the file properly: {file_path}")
    return df

# Function to convert a DataFrame chunk into a GeoDataFrame
def convert_to_gdf(df, lat_col='latitude', long_col='longitude', crs='EPSG:4326'):
    geometry = [Point(xy) for xy in zip(df[long_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=crs)
    return gdf

# Function to merge two GeoDataFrames by nearest join on geometry and date
def merge_gdfs_nearest_by_date(gdf_scalar_chunk, gdf_density_chunk):
    # Perform spatial join where we merge nearest points (left join)
    merged_gdf = gpd.sjoin_nearest(gdf_scalar_chunk, gdf_density_chunk, how='left', distance_col='distance')
    return merged_gdf

# Process each date separately and save the result
def process_date_chunk(args):
    date, gdf_scalar_chunk, gdf_density_by_date, output_folder = args

    # Get the density data for the same date
    if date in gdf_density_by_date.groups:
        gdf_density_chunk = gdf_density_by_date.get_group(date)
        
        # Convert chunks to GeoDataFrames
        gdf_scalar_gdf = convert_to_gdf(gdf_scalar_chunk)
        gdf_density_gdf = convert_to_gdf(gdf_density_chunk)
        
        gdf_scalar_gdf = gdf_scalar_gdf.to_crs(epsg=3857)
        gdf_density_gdf = gdf_density_gdf.to_crs(epsg=3857)

        # Perform the nearest merge
        merged_gdf = merge_gdfs_nearest_by_date(gdf_scalar_gdf, gdf_density_gdf)
        
        # Save the result as a CSV file with the date as the filename
        output_file = os.path.join(output_folder, f"{date}.csv")
        merged_gdf.to_csv(output_file, index=False)
        print(f"Saved merged data for {date} to {output_file}")
    else:
        print(f"No matching density data for date {date}")

# Main process
if __name__ == "__main__":
    # Load datasets as regular DataFrames
    df_density = load_data('fake_density_data_parallel.csv')  # 1x1 resolution
    df_scalar = load_data('fake_scalar_data_parallel_0.5.csv')  # 0.5x0.5 resolution

    
    
    # Group both datasets by date
    df_density_by_date = df_density.groupby('date')
    df_scalar_by_date = df_scalar.groupby('date')

    # Prepare output folder for the results
    output_folder = "merged_results"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Prepare date groups for parallel processing
    scalar_date_groups = list(df_scalar_by_date)

    # Use multiprocessing to process each date group in parallel
    num_cores = 4
    with Pool(num_cores) as pool:
        pool.map(process_date_chunk, [(date, gdf_scalar_chunk, df_density_by_date, output_folder) for date, gdf_scalar_chunk in scalar_date_groups])

    print("Merging process completed.")


Trying to read the file  fake_density_data_parallel.csv
We have read the file properly: fake_density_data_parallel.csv
Trying to read the file  fake_scalar_data_parallel_0.5.csv
We have read the file properly: fake_scalar_data_parallel_0.5.csv
Saved merged data for 2023-01-01 to merged_results/2023-01-01.csv
Saved merged data for 2023-01-24 to merged_results/2023-01-24.csv
Saved merged data for 2023-02-16 to merged_results/2023-02-16.csv
Saved merged data for 2023-03-11 to merged_results/2023-03-11.csv
Saved merged data for 2023-01-02 to merged_results/2023-01-02.csv
Saved merged data for 2023-01-25 to merged_results/2023-01-25.csv
Saved merged data for 2023-02-17 to merged_results/2023-02-17.csv
Saved merged data for 2023-03-12 to merged_results/2023-03-12.csv
Saved merged data for 2023-01-03 to merged_results/2023-01-03.csv
Saved merged data for 2023-01-26 to merged_results/2023-01-26.csv
Saved merged data for 2023-02-18 to merged_results/2023-02-18.csv
Saved merged data for 2023-03-

In [ ]:
!pip install pygeos
!pip install shapely
!pip install shapely --upgrade

In [1]:
import pygeos
import shapely
import geopandas as gpd

print(f"PyGEOS version: {pygeos.__version__}")
print(f"Shapely version: {shapely.__version__}")
print(f"GeoPandas version: {gpd.__version__}")


PyGEOS version: 0.14
Shapely version: 2.0.6
GeoPandas version: 0.14.4
